In [3]:
import csv
import numpy as np
import math
import sys

#read data

data = []
# 每一個維度儲存一種污染物的資訊
for i in range(18):
	data.append([])

n_row = 0
text = open('train.csv', 'r', encoding='big5') 
row = csv.reader(text , delimiter=",")
for r in row:
    # 第0列沒有資訊
    if n_row != 0:
        # 每一列只有第3-27格有值(1天內24小時的數值)
        for i in range(3,27):
            if r[i] != "NR":
                data[(n_row-1)%18].append(float(r[i]))
            else:
                data[(n_row-1)%18].append(float(0))	
    n_row = n_row+1

text.close()

#parse data to (x,y)

x = []
y = []
# 每 12 個月
for i in range(12):
    # 一個月取連續10小時的data可以有471筆
    for j in range(471):
        x.append([])
        # 18種污染物
        for t in range(18):
            # 連續9小時
            for s in range(9):
                x[471*i+j].append(data[t][480*i+j+s] )
        y.append(data[9][480*i+j+9])
x = np.array(x)
y = np.array(y)

# add square term
# x = np.concatenate((x,x**2), axis=1)

# add bias
x = np.concatenate((np.ones((x.shape[0],1)),x), axis=1)

#init weight & other hyperparams
w = np.zeros(len(x[0]))
l_rate = 10
repeat = 10000

#check your ans with close form solution
# use close form to check whether ur gradient descent is good
# however, this cannot be used in hw1.sh 
# w = np.matmul(np.matmul(inv(np.matmul(x.transpose(),x)),x.transpose()),y)

#start training 
x_t = x.transpose()
s_gra = np.zeros(len(x[0]))

for i in range(repeat):
    hypo = np.dot(x,w) #dot	函數在一維向量為內稽，二維時為正常矩陣相乘(與*相同);multiply則為entry相乘
    loss = hypo - y
    cost = np.sum(loss**2) / len(x)
    cost_a  = math.sqrt(cost)
    gra = np.dot(x_t,loss)
    s_gra += gra**2
    ada = np.sqrt(s_gra)
    w = w - l_rate * gra/ada
    #print ('iteration: %d | Cost: %f  ' % ( i,cost_a))

# save model
np.save('model.npy',w)
# read model
w = np.load('model.npy')

#read testing data
test_x = []
n_row = 0
text = open('test.csv' ,"r")
row = csv.reader(text , delimiter= ",")
print(w)
for r in row:
    if n_row %18 == 0:
        test_x.append([])
        for i in range(2,11):
            test_x[n_row//18].append(float(r[i]) )
    else :
        for i in range(2,11):
            if r[i] !="NR":
                test_x[n_row//18].append(float(r[i]))
            else:
                test_x[n_row//18].append(0)
    n_row = n_row+1
text.close()
test_x = np.array(test_x)

# add square term
# test_x = np.concatenate((test_x,test_x**2), axis=1)

# add bias
test_x = np.concatenate((np.ones((test_x.shape[0],1)),test_x), axis=1)

#get ans.csv with your model
ans = []
for i in range(len(test_x)):
    ans.append(["id_"+str(i)])
    a = np.dot(w,test_x[i])
    ans[i].append(a)

filename = "predict.csv"
text = open(filename, "w+")
s = csv.writer(text,delimiter=',',lineterminator='\n')
s.writerow(["id","value"])
for i in range(len(ans)):
    s.writerow(ans[i]) 
text.close()

[ 2.54076977e-02 -2.20947067e-02 -5.50897814e-03  4.04126682e-02
  9.05866323e-03  1.74051999e-02 -9.41232449e-04 -2.68493029e-02
 -3.54069918e-02 -2.10050620e-02  1.21122768e-02 -2.73389556e-02
 -2.60922920e-02  2.29273428e-02  6.53181125e-03 -2.10707907e-03
  2.91086251e-02  6.42478041e-02  2.77075849e-02 -1.38720696e-01
 -2.37962637e-01 -1.57966718e-01 -1.42317012e-01 -1.38791079e-01
 -4.36312624e-02  4.68854071e-02  1.49516305e-01  1.15969854e-01
 -1.34255742e-01 -4.93326575e-01 -3.13498762e-01 -2.81742461e-01
 -1.22826399e-01 -1.83664390e-01 -1.65509939e-03  2.76174575e-01
  4.13244252e-01 -1.12775862e-01 -3.65080239e-02  1.62418053e-01
 -1.19620752e-02 -1.39167213e-02  4.89505049e-02  6.29431025e-02
 -1.74752725e-01  4.88496098e-02  6.12614431e-02 -3.38566670e-02
 -5.84991350e-02 -2.04376688e-02 -3.04712481e-02 -8.15278646e-02
  5.46059541e-02  1.04310338e-01  4.31145795e-02  1.11291629e-02
 -2.74800330e-02 -1.76782752e-02 -1.97107480e-02 -1.99327326e-02
 -5.69285051e-02  4.84815